# Install tqdm library

In [ ]:
# Supressing the warning messages
import warnings
warnings.filterwarnings('ignore')
!pip3 install tqdm

In [ ]:
import pandas as pd
import numpy as np
DHP=pd.read_csv('ResTracker Export_Filtered_ANN_Train&Test_Updated.csv', encoding='latin')
print('Shape before deleting duplicate values:',DHP.shape)
DHP_rename = DHP.rename(columns={'Latitude':'Lat','Longitude':'Long','Gas Rate (TJ/d)':'Gas Rate(TJ/d)', 'Water Rate (m3/d)':'Water Rate(m3/d)','Cumulative Gas(bcf)':'Cum Gas(bcf)','Cumulative Water(10e3 m3)':'Cum Water(10e3 m3)','Cumulative Online Days':'Cum Online Days','Casing Pressure (kPag)':'CP(kPag)','Downhole Pressure (psi)':'BHP(psi)','Depth_Argle_m':'Depth(m)'})
# Removing duplicate rows if any
DHP_rename = DHP_rename.drop_duplicates()
print('Shape After deleting duplicate values:', DHP.shape)

# Visualize the dataset variables

In [ ]:
%matplotlib inline
# Creating Bar chart as the Target variable is Continuous
DHP_rename.hist(['Gas Rate(TJ/d)','Water Rate(m3/d)','Cum Gas(bcf)','Cum Water(10e3 m3)','Cum Online Days','CP(kPag)','BHP(psi)','Depth(m)'],figsize=(20,10))

# Creating Bar chart for the Target variable as It is Continuous Value

In [ ]:
%matplotlib inline
# Creating Bar chart as the Target variable is Continuous
DHP_rename['BHP(psi)'].hist()

# Import required libraries

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LinearRegression
from sklearn. linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor

from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn import metrics 

# Check the dataset information details

In [ ]:
DHP_rename.info()

# Data Description

In [ ]:
DHP_rename.describe(include='all').T

# Remove the Outliers of BHP based on domain knowledge

In [ ]:
#Remove the high DHP
DHP_rename_Remove_DHP=DHP_rename[DHP_rename['BHP(psi)']<450]
#DHP_Data.describe(include='all').T.to_excel("Statistics_Table.xlsx")
DHP_rename_Remove_DHP.describe(include='all').T

# Remove the Outliers of High Casing Pressure based on Domain Knowledge

In [ ]:
DHP_rename_Remove_DHP_Case.describe(include='all').T

In [ ]:
# select the numeric data from D1 dataset for variable distribution analysis and identify the data quality as well
DHP_rename_Remove_DHP_Case_numeric = DHP_rename_Remove_DHP_Case.select_dtypes(include=[np.number]).iloc[:,:]
DHP_rename_Remove_DHP_Case_vars = list(DHP_rename_Remove_DHP_Case.columns.values)

# Define the function to plot all the numerical columns to better understand data distributions post data-processing

In [ ]:
# define the function to plot all the numerical columns to help understand data distributions 
def check_numeric_var(var, name):
    print('\n>>>>>>>>>>', name, '<<<<<<<<<<\n')
    print(var.describe())
    print("\nSkew is", round(var.skew(axis=None, skipna=True),4))
    fig = plt.figure(figsize=(16,6))
    plt.subplot(1,2,1)
    hist_plot = sns.distplot(var) # plotting the histogram plot for data distribution
    plt.subplot(1,2,2)
    box_plot= sns.boxplot(var) #plotting the boxplot for data quality check
    plt.show()
    x = 0 # go thru all the columns from DHP dataset
    while x < 28:  
        print('&', end=" ")
        x = x + 1 
    print('\n'); 

In [ ]:
# apply the define function to plot the distribution and boxplot for numerical data columns in D1
for n in DHP_rename_Remove_DHP_Case_vars:
    check_numeric_var(DHP_rename_Remove_DHP_Case[n],n)

In [ ]:
DHP_rename_Remove_DHP_Case.hist(['Gas Date','Gas Rate(TJ/d)','Water Rate(m3/d)','Lat','Long','Cum Gas(bcf)', 'Cum Water(10e3 m3)', 'Cum Online Days','CP(kPag)','Depth(m)','DHP(psi)'],figsize=(20,15))

# Seaborn Joint Plots between Variables and Target

In [ ]:
ContinuousCols=['Gas Date', 'Gas Rate(TJ/d)', 'Water Rate(m3/d)', 'Lat', 'Long', 'Cum Gas(bcf)', 'Cum Water(10e3 m3)', 'Cum Online Days',
                 'CP(kPag)','Depth(m)']

# Plotting scatter chart for each predictor vs the target variable
for predictor in ContinuousCols:
 #   DHP_Remove_DHP_Case.plot.scatter(x=predictor, y='DHP (psi)', figsize=(10,5), title=predictor+" VS "+ 'DHP (psi)')

    g = sns.jointplot(x=predictor,y='BHP(psi)',data =DHP_rename_Remove_DHP_Case,ratio=2,scatter_kws={'s': 3, 'linewidth': 0.1},marginal_kws=dict(fill=False),kind="reg",joint_kws={'color':'blue','ci':True})
    regline = g.ax_joint.get_lines()[0]
    regline.set_color('red')
    regline.set_zorder(1)

# Calculate the skew and kurtosis for the variables

In [ ]:
# Calculate the skewness of the dataset
DHP_rename_Remove_DHP_Case.skew(axis=0,skipna=True)

In [ ]:
# Calculate the skewness of the dataset
DHP_rename_Remove_DHP_Case.kurtosis(axis=0,skipna=True)

# Examining correlation matrix using heatmap

In [ ]:
#cmap = sns.diverging_palette(205, 133, 63, as_cmap=True)
#cols = (["#682F2F", "#9E726F", "#D6B2B1", "#B9C0C9", "#9F8A78", "#F3AB60"])
corrmat= DHP_rename_Remove_DHP_Case.corr()
sns.set(font_scale=1.2)
f, ax = plt.subplots(figsize=(14,12))
# create a mask
mask = np.zeros_like(corrmat, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
np.fill_diagonal(mask, False)
# create heatmap
sns.heatmap(corrmat,mask=mask,center=0,cmap="coolwarm",annot=True)
plt.xticks(rotation=45) 
plt.savefig("heat_map.jpg")

# Examine the kde pairplots 

In [ ]:
# DHP_rename_Remove_DHP_Case_Drop = DHP_rename_Remove_DHP_Case.drop(['Lat','Long'], axis=1)
g=sns.PairGrid(DHP_rename_Remove_DHP_Case,hue='Status')
g.map_diag(plt.hist)
#g.map_offdiag(sns.scatterplot, size=DHP_rename_Remove_DHP_Case_Drop['Status'])
#g.map_offdiag(sns.scatterplot)
g.map_upper(sns.kdeplot,levels=5,color='0.2')
g.map_lower(sns.scatterplot)
g.add_legend(title='',adjust_subtitles=True)
plt.savefig("Pairplot.jpg")

# Export the preprocessing data for ANN model

In [ ]:
# DHP_rename_Remove_DHP_Case_Drop_more = DHP_rename_Remove_DHP_Case.drop(['Lat','Long','Status','Pumping_Status'], axis=1)
# Remove the Lat and Long for another exercise
DHP_rename_Remove_DHP_Case_Drop_more = DHP_rename_Remove_DHP_Case.drop(['Status','Pumping_Status'], axis=1)
DHP_rename_Remove_DHP_Case_Drop_more.to_csv("ResTracker Export after preprocessing_V1.csv")